In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers/S20_tokenizer"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers/S20_tokenizer


In [5]:
import urllib.request
url = "https://raw.githubusercontent.com/shubham721/Sentiment-Analysis-On-Hindi-Reviews/master/pos_hindi.txt"
url = "https://raw.githubusercontent.com/shubham721/Sentiment-Analysis-On-Hindi-Reviews/master/neg_hindi.txt"
filename = "data/hindi_neg.txt"
urllib.request.urlretrieve(url, filename)

('data/hindi_neg.txt', <http.client.HTTPMessage at 0x7eff6cc164a0>)

In [6]:
# Define the file paths
file1_path = "hindi_positive.txt"
file2_path = "hindi_neg.txt"
combined_file_path = "hindi_combined.txt"

file_path1 = os.path.join(os.getcwd(), "data", file1_path)
file_path2 = os.path.join(os.getcwd(), "data", file2_path)
combined_file_path = os.path.join(os.getcwd(), "data", combined_file_path)

# Read the contents of the first file
with open(file_path1, "r") as file1:
    content1 = file1.read()

# Read the contents of the second file
with open(file_path2, "r") as file2:
    content2 = file2.read()

# Combine the contents
combined_content = content1 + "\n" + content2

# Write the combined contents to a new file
with open(combined_file_path, "w") as combined_file:
    combined_file.write(combined_content)

print(
    f"Contents of {file1_path} and {file2_path} have been combined into {combined_file_path}."
)

Contents of hindi_positive.txt and hindi_neg.txt have been combined into /mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers/S20_tokenizer/data/hindi_combined.txt.


In [8]:
# reading the input file
combined_file_path = "hindi_combined.txt"
combined_file_path = os.path.join(os.getcwd(), "data", combined_file_path)
with open(combined_file_path, "r") as file:
    combined_content = file.read()

In [9]:
print(f"The combined file contains {len(combined_content.splitlines())} lines.")

The combined file contains 1186 lines.


In [10]:
from tokenizer.basic_bpe import BasicTokenizer
from tokenizer.regex_bpe import RegexTokenizer

In [11]:
basic_tokenizer = BasicTokenizer()
basic_tokenizer.train(combined_content, vocab_size=5000, verbose=True)

merge 1/4744: (224, 164) -> 256 (b'\xe0\xa4') had 48775 occurrences
merge 2/4744: (32, 256) -> 257 (b' \xe0\xa4') had 15209 occurrences
merge 3/4744: (224, 165) -> 258 (b'\xe0\xa5') had 14761 occurrences
merge 4/4744: (256, 190) -> 259 (b'\xe0\xa4\xbe') had 4691 occurrences
merge 5/4744: (258, 135) -> 260 (b'\xe0\xa5\x87') had 3746 occurrences
merge 6/4744: (256, 176) -> 261 (b'\xe0\xa4\xb0') had 3693 occurrences
merge 7/4744: (258, 128) -> 262 (b'\xe0\xa5\x80') had 3077 occurrences
merge 8/4744: (258, 141) -> 263 (b'\xe0\xa5\x8d') had 2822 occurrences
merge 9/4744: (257, 149) -> 264 (b' \xe0\xa4\x95') had 2802 occurrences
merge 10/4744: (256, 191) -> 265 (b'\xe0\xa4\xbf') had 2597 occurrences
merge 11/4744: (256, 130) -> 266 (b'\xe0\xa4\x82') had 2590 occurrences
merge 12/4744: (259, 256) -> 267 (b'\xe0\xa4\xbe\xe0\xa4') had 2407 occurrences
merge 13/4744: (263, 256) -> 268 (b'\xe0\xa5\x8d\xe0\xa4') had 2229 occurrences
merge 14/4744: (265, 256) -> 269 (b'\xe0\xa4\xbf\xe0\xa4') had 20

In [12]:
model_path = os.path.join(os.getcwd(), "tokenizer_model")
os.makedirs(model_path, exist_ok=True)
prefix = os.path.join(model_path, "hindi_sentiments_basic")
basic_tokenizer.save(prefix)

In [13]:
model_path = os.path.join(os.getcwd(), "tokenizer_model")
model_path = os.path.join(model_path, "hindi_sentiments_basic.model")

In [15]:
basic_tokenizer = BasicTokenizer()
basic_tokenizer.load(model_path)

In [17]:
ids  = basic_tokenizer.encode("आपका सुरूर और कर्ज के मुकाबले हिमेश इस बार रोल में जमे है")
print(ids)

[256, 1330, 4598, 735, 553, 812, 1106, 1523, 1377, 4601, 4602, 2997, 358]


In [18]:
decoded = basic_tokenizer.decode(ids)
print(decoded)

आपका सुरूर और कर्ज के मुकाबले हिमेश इस बार रोल में जमे है


In [22]:
def test_tokenizer(text):
    ids = basic_tokenizer.encode(text)
    decoded = basic_tokenizer.decode(ids)
    mapping = [(str(i), basic_tokenizer.decode([i])) for i in ids]

    return ids, decoded, mapping

In [23]:
import gradio as gr

In [24]:
with gr.Blocks() as demo:
    gr.HTML("<h1 align = 'center'> Token Generation for Hindi Dataset </h1>")

    with gr.Row():
      with gr.Column():
        inputs = [gr.TextArea(label = "Enter initial text to generate tokens in Hindi", lines = 10)]
        generate_btn = gr.Button(value = 'Generate Text')
      with gr.Column():
        enc = gr.Textbox(label = "Encoded Tokens")
        txt = gr.Textbox(label = "Decoded Text from tokens")
        map = gr.Textbox(label = "Mapping of the tokens and respective texts")
        outputs  = [
            enc,
            txt,
            map
            ]
    
    generate_btn.click(fn = test_tokenizer, inputs= inputs, outputs = outputs)


In [26]:
demo.launch(share=True, inbrowser=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://d4a63996007b8969ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [27]:
demo.close()

Closing server running on port: 7860
